<a href="https://colab.research.google.com/github/kyle-woodward/EEpythonNotebooks/blob/main/ExampleEEificationAndDeploying.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## added .json dictionary of required model details to use in model inference
Example notebook on loading a saved model and working through the steps to EE-ify the model and deplay to the AI platform for inferance in GEE.

There have been some minor updates since the first pass of notebooks we have used for this process which are reflected in the current example of hosting a [Hostable DNN for prediction in Earth Engine](https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_TensorFlow_AI_Platform.ipynb).



# Authentication steps

In [ ]:
import os
# Cloud authentication.
from google.colab import auth
auth.authenticate_user()


In [ ]:
# Import, authenticate and initialize the Earth Engine library.
import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=JjrWvHoUx2mqtWeUw3oaIjLbfaWacZF2xiR7awE5aic&tc=EMsBQP0Z3vdJcxbdUckqPiH0pOIAfRFEAPKBppFrmn4&cc=4Lc41eb4-_pjuOJEBc-akwTaI7o_zKCCChn5W9femXA

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhgPAQZZvBo4EFhVqgTwpqrvKsXN230e72qLWBIdPPw8Zs1aw2u0K0

Successfully saved authorization token.


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K

In [ ]:
from dataclasses import dataclass
import time


@dataclass
class model:
  name: str
  bucket:str
  folder:str
  project: str
  

# Load a saved model

Example loads a saved keras model in .h5 format. A TF dir could be loaded in the same manner as well.

In [ ]:
model = model(
    name = 'vgg16unet_model_kdw_sigmoid_mae_v18_rev2', # without .h5 file extension (gets added in later)
    bucket = 'landfire',
    folder = 'lucas/pools2fire_tf',
    project = 'pyregence-ee'
)

# MODEL_NAME = 'vgg16unet_model_kdw_linear_huber_delta0_5_v5.h5'
# BUCKET = 'landfire'
# FOLDER = 'lucas/pools2fire_tf'

model_path =  f"gs://{model.bucket}/{model.folder}/{model.name}.h5"
# model_path =  f"gs://{BUCKET}/{FOLDER}/{MODEL_NAME}"
# model.save(f"{MODEL_NAME}.h5")
new_model = tf.keras.models.load_model(model_path)

# Check its architecture
new_model.summary()

Model: "vgg16-unet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None, None,  0           []                               
                                 18)]                                                             
                                                                                                  
 block1_conv1 (Conv2D)          (None, None, None,   10432       ['input[0][0]']                  
                                64)                                                               
                                                                                                  
 block1_conv2 (Conv2D)          (None, None, None,   36928       ['block1_conv1[0][0]']           
                                64)                                                      

#Save the trained model

Export the trained model to TensorFlow SavedModel format in your cloud storage bucket. The Cloud Platform storage browser is useful for checking on these saved models. (taken directly from Hostable DNN for prediction in Earth Engine example).



Note: do we need to save it as an 'estimator' TF DIR? or could we keep it in the .h5 format?

 A. I think so... get_meta_graph_def(TF_FIR) reads the graph from a directory. 
 

In [ ]:
#  save the model as a TF Estimator which is what 
model.estimator = model.name.split('.')[0] + '_ESTIMATOR'
model.tf_dir = 'gs://{}/{}/{}'.format(model.bucket,model.folder,model.estimator)
print(model.tf_dir)
tf.keras.models.save_model(new_model,model.tf_dir,save_format='tf')


gs://landfire/lucas/pools2fire_tf/vgg16unet_model_kdw_sigmoid_mae_v18_rev2_ESTIMATOR
INFO:tensorflow:Assets written to: gs://landfire/lucas/pools2fire_tf/vgg16unet_model_kdw_sigmoid_mae_v18_rev2_ESTIMATOR/assets


# EEification

EEIfication prepares the model for hosting on [Google AI Platform](https://cloud.google.com/ai-platform).  Learn more about EEification from [this doc](https://developers.google.com/earth-engine/tensorflow#interacting-with-models-hosted-on-ai-platform).  First, get (and SET) input and output names of the nodes.  **CHANGE THE OUTPUT NAME TO SOMETHING THAT MAKES SENSE FOR YOUR MODEL!**  Keep the input name of 'array', which is how you'll pass data into the model (as an array image).

In [ ]:
from tensorflow.python.tools import saved_model_utils
model.output = 'pools2BP'

meta_graph_def = saved_model_utils.get_meta_graph_def(model.tf_dir, 'serve')
inputs = meta_graph_def.signature_def['serving_default'].inputs
outputs = meta_graph_def.signature_def['serving_default'].outputs

# Just get the first thing(s) from the serving signature def.  i.e. this
# model only has a single input and a single output.
input_name = None
for k,v in inputs.items():
    input_name = v.name
    break

output_name = None
for k,v in outputs.items():
    output_name = v.name
    break

# Make a dictionary that maps Earth Engine outputs and inputs to 
# AI Platform inputs and outputs, respectively.
import json
input_dict = "'" + json.dumps({input_name: "array"}) + "'"
output_dict = "'" + json.dumps({output_name: model.output}) + "'"
print(input_dict)
print(output_dict)

'{"serving_default_input:0": "array"}'
'{"StatefulPartitionedCall:0": "pools2BP"}'


## Run the EEifier

The actual EEification is handled by the `earthengine model prepare` command.  Note that you will need to set your Cloud Project prior to running the command.

In [ ]:
# Put the EEified model next to the trained model directory.
model.region = 'us-central1'
model.eeified_dir = 'gs://{}/{}/eeified_{}/'.format(model.bucket,model.folder,model.estimator)
# change to your specific project
#PROJECT = 'pyregence-ee'
#REGION = 'us-central1'
# # You need to set the project before using the model prepare command.
!earthengine set_project {model.project}
!earthengine model prepare --source_dir {model.tf_dir} --dest_dir {model.eeified_dir} --input {input_dict} --output {output_dict}

Successfully saved project id
Success: model at 'gs://landfire/lucas/pools2fire_tf/eeified_vgg16unet_model_kdw_sigmoid_mae_v18_rev2_ESTIMATOR/' is ready to be hosted in AI Platform.


# Deploy and host the EEified model on AI Platform

Now there is another TensorFlow `SavedModel` stored in `EEIFIED_DIR` ready for hosting by AI Platform.  Do that from the `gcloud` command line tool, installed in the Colab runtime by default.  Be sure to specify a regional model with the `REGION` parameter.  Note that the `MODEL_NAME` must be unique.  If you already have a model by that name, either name a new model or a new version of the old model.  The [Cloud Console AI Platform models page](https://console.cloud.google.com/ai-platform/models) is useful for monitoring your models.

**If you change anything about the trained model, you'll need to re-EEify it and create a new version!**

In [ ]:
!gcloud ai-platform models create {model.estimator} --project {model.project} --region {model.region}

Using endpoint [https://us-central1-ml.googleapis.com/]
Created ai platform model [projects/pyregence-ee/models/vgg16unet_model_kdw_sigmoid_mae_v18_rev2_ESTIMATOR].


In [ ]:
import time

#MODEL_NAME = MODEL_NAME_ESTIMATOR
model.version = 'v' + str(int(time.time()))
print('Creating version: ' + model.version)

!gcloud ai-platform versions create {model.version} \
  --project {model.project} \
  --region {model.region} \
  --model {model.estimator} \
  --origin {model.eeified_dir} \
  --framework "TENSORFLOW" \
  --runtime-version=2.3 \
  --python-version=3.7

Creating version: v1651845684
Using endpoint [https://us-central1-ml.googleapis.com/]


In [ ]:
print('MODEL_NAME=',model.name)
print('VERSION_NAME=',model.version)
print('PROJECT=',model.project)

MODEL_NAME= vgg16unet_model_kdw_sigmoid_mae_v18_rev2
VERSION_NAME= v1651845684
PROJECT= pyregence-ee


# export all model eeification params to a .json to use for inference

In [ ]:
model.__dict__

{'bucket': 'landfire',
 'eeified_dir': 'gs://landfire/lucas/pools2fire_tf/eeified_vgg16unet_model_kdw_sigmoid_mae_v18_rev2_ESTIMATOR/',
 'estimator': 'vgg16unet_model_kdw_sigmoid_mae_v18_rev2_ESTIMATOR',
 'folder': 'lucas/pools2fire_tf',
 'name': 'vgg16unet_model_kdw_sigmoid_mae_v18_rev2',
 'output': 'pools2BP',
 'project': 'pyregence-ee',
 'region': 'us-central1',
 'tf_dir': 'gs://landfire/lucas/pools2fire_tf/vgg16unet_model_kdw_sigmoid_mae_v18_rev2_ESTIMATOR',
 'version': 'v1651845684'}

In [ ]:
from google.cloud import storage
import json

def upload_blob(bucket_name, source_file_name, destination_blob_name):
  """Uploads a file to the bucket."""
  # TODO : add project name to tf_run or tf_samples
  storage_client = storage.Client('pyregence')
  bucket = storage_client.get_bucket(bucket_name)
  blob = bucket.blob(destination_blob_name)

  blob.upload_from_filename(source_file_name)

  print('File {} uploaded to {}.'.format(
      source_file_name,
      destination_blob_name))
  
model_file_name = f"{model.name.split('.')[0]}.json"
model_file_dest = f"{model.folder}/{model_file_name}"
print(model_file_name)
print(model_file_dest)
with open(model_file_name, 'w') as fp:
    json.dump(model.__dict__, fp)

upload_blob(model.bucket, model_file_name, model_file_dest)

vgg16unet_model_kdw_sigmoid_mae_v18_rev2.json
lucas/pools2fire_tf/vgg16unet_model_kdw_sigmoid_mae_v18_rev2.json
File vgg16unet_model_kdw_sigmoid_mae_v18_rev2.json uploaded to lucas/pools2fire_tf/vgg16unet_model_kdw_sigmoid_mae_v18_rev2.json.
